In [3]:
# Import the libraries
from typing import List, Tuple

import pandas as pd
import numpy as np

## Baseline Model

For the baseline model, implement a normal `Decision Tree` classifier

### Data Proprocessing (if any)

In [4]:
df_train = pd.read_csv('train.csv')
df_val = pd.read_csv('val.csv')
df_train.columns

Index(['Party', 'Feature-1', 'Feature-2', 'Feature-3', 'Feature-4',
       'Feature-5', 'Feature-6', 'Feature-7', 'Feature-8', 'Feature-9',
       'Feature-10', 'Feature-11', 'Feature-12', 'Feature-13', 'Feature-14',
       'Feature-15', 'Feature-16'],
      dtype='object')

In [5]:
def preprocess_data(df):
    # Convert 'y' and 'n' to binary values (1 and 0)
    df = df.replace({'y': 1, 'n': 0})

    # Map "democrat" to 1 and "republican" to 0
    df['Party'] = df['Party'].map({'democrat': 0, 'republican': 1})

    # Split the data into features and labels
    X = df.drop('Party', axis=1).values
    y = df['Party'].values

    return X, y

Implement the Decision Tree class below.

In [6]:
from collections import Counter
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

Train the model on the training data and report the accuracy and [F1 score](https://en.wikipedia.org/wiki/F-score) on the validation data.

In [7]:
X_train, y_train = preprocess_data(df_train)
X_val, y_val = preprocess_data(df_val)

# Initialize and train the DecisionTree model
clf = DecisionTree(max_depth=9)
clf.fit(X_train, y_train)
predictions = clf.predict(X_val)

# Define a function to calculate accuracy
def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

# Define a function to calculate F1 score
def calculate_f1_score(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

# Make predictions on the validation data
y_val_pred = clf.predict(X_val)

# Calculate accuracy and F1 score
accuracy = accuracy(y_val, y_val_pred)
f1 = calculate_f1_score(y_val, y_val_pred)

print(f"Accuracy on validation data: {accuracy}")
print(f"F1 Score on validation data: {f1}")

Accuracy on validation data: 0.9090909090909091
F1 Score on validation data: 0.8799999999999999


## Bagging

In [8]:
def get_bootstrap_samples(df: pd.DataFrame, n_samples: int, sample_fraction: float, target: str) -> List[pd.DataFrame]:
    """Generate bootstrap samples using the given dataframe.
    """
    bootstrap_samples = []

    for _ in range(n_samples):
        # Randomly select row indices with replacement
        sample_indices = np.random.choice(df.index, size=int(len(df) * sample_fraction), replace=True)

        # Create a bootstrap sample by selecting rows based on the sampled indices, including the target variable
        bootstrap_sample = df.loc[sample_indices]

        # Append the bootstrap sample to the list
        bootstrap_samples.append(bootstrap_sample)

    return bootstrap_samples

In [9]:
n_trees = 10
sample_fraction = 0.8
target = 'Party' # TODO: Set the target column
trees = []

bootstrap_samples = get_bootstrap_samples(df_train, n_trees, sample_fraction, target)

# Write your code below
for bootstrap_sample in bootstrap_samples:
    # Replace 'y' with 1 and 'n' with 0 in both features (X) and target (y)
    bootstrap_sample = bootstrap_sample.replace({'y': 1, 'n': 0, 'republican': 1, 'democrat': 0})

    # Separate features (X) and target (y)
    X = bootstrap_sample.drop(target, axis=1)
    y = bootstrap_sample[target]

    # Initialize and train a DecisionTree model
    tree = DecisionTree(max_depth=5)
    tree.fit(X.values, y.values)

    # Append the trained tree to the 'trees' list
    trees.append(tree)

In [10]:
X_val, y_val = preprocess_data(df_val)

# Initialize lists to store predictions from each tree
all_tree_predictions = []

# Make predictions for each data point using each trained tree
for tree in trees:
    tree_predictions = tree.predict(X_val)
    all_tree_predictions.append(tree_predictions)

# Combine predictions using majority voting
combined_predictions = np.mean(np.array(all_tree_predictions), axis=0) >= 0.5

# Calculate accuracy and F1 Score
correct_predictions = np.sum(combined_predictions == y_val)
total_samples = len(y_val)
accuracy = correct_predictions / total_samples

# Calculate precision, recall, and F1 Score
true_positives = np.sum((combined_predictions == 1) & (y_val == 1))
false_positives = np.sum((combined_predictions == 1) & (y_val == 0))
false_negatives = np.sum((combined_predictions == 0) & (y_val == 1))

precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)

# Calculate F1 Score
f1_score = 2 * (precision * recall) / (precision + recall)

# Print accuracy and F1 Score
print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1_score:.2f}")

Accuracy: 0.94
F1 Score: 0.92


## Random Forest Algorithm

In [11]:
def get_random_forest_bootstrap_samples(df: pd.DataFrame, n_samples: int, sample_fraction: float, feature_fraction: float, target: str) -> Tuple[List[pd.DataFrame], List[str]]:
    """Generate bootstrap samples using the given dataframe.
    """
    bootstrap_samples = []
    selected_features_list = []

    for _ in range(n_samples):
        # Randomly select row indices with replacement
        sample_indices = np.random.choice(df.index, size=int(len(df) * sample_fraction), replace=True)

        # Randomly select a subset of features
        num_features = int(len(df.columns) * feature_fraction)
        selected_features = np.random.choice(df.columns.drop(target), size=num_features, replace=False)
        selected_features_list.append(selected_features.tolist())

        # Create a bootstrap sample by selecting rows and features based on the sampled indices
        bootstrap_sample = df.loc[sample_indices, [target] + selected_features.tolist()]

        # Append the bootstrap sample to the list
        bootstrap_samples.append(bootstrap_sample)

    return bootstrap_samples, selected_features_list

In [12]:
n_trees = 10
sample_fraction = 0.8
feature_fraction = 0.7
target = 'Party' # TODO: Set the target column
trees = []

bootstrap_samples, sample_features = get_random_forest_bootstrap_samples(df_train, n_trees, sample_fraction, feature_fraction, target)

# Write your code below
for bootstrap_sample, selected_features in zip(bootstrap_samples, sample_features):
    # Replace 'y' with 1, 'n' with 0, 'republican' with 1, and 'democrat' with 0 in both features (X) and target (y)
    bootstrap_sample = bootstrap_sample.replace({'y': 1, 'n': 0, 'republican': 1, 'democrat': 0})

    # Separate features (X) and target (y)
    X = bootstrap_sample[selected_features]
    y = bootstrap_sample[target]

    # Initialize and train a DecisionTree model
    tree = DecisionTree(min_samples_split=2, max_depth=5, n_features=None)
    tree.fit(X.values, y.values)

    # Append the trained tree to the 'trees' list
    trees.append(tree)



## Prediction on Test Data

In [13]:
df_test = pd.read_csv('test.csv')

In [14]:
df_test = df_test.replace({'y': 1, 'n': 0})

# Initialize an empty list to store individual Decision Tree predictions
individual_predictions = []

# Make predictions on the test data using each Decision Tree
for tree in trees:
    X_test = df_test[sample_features[0]]  # Use the same selected features as the training data
    tree_predictions = tree.predict(X_test.values)
    individual_predictions.append(tree_predictions)

# Combine predictions using majority voting
combined_predictions = np.mean(np.array(individual_predictions), axis=0) >= 0.5

# Create a DataFrame with 'ID' and 'Party' columns
submission_df = pd.DataFrame({'ID': df_test['ID'], 'Party': combined_predictions.astype(int)})

# Save the predictions to a CSV file
submission_df.to_csv('submission.csv', index=False)